In [103]:
!pip install scanpy scipy umap-learn leidenalg
!pip install hmmlearn
!pip install scikit-learn --quiet

# Set up
We will need to first import several necessary packages and load the data. The data needs to follow normalization steps: total count normalization (target sum = 1e4) and log1p, perform PCA, compute a neighborhood graph, embed with UMAP, and finally compute Leiden clusters so that our downstream CNA detection functions have properly clustered input.

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import scipy.sparse
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_recall_fscore_support
from hmmlearn import hmm
from scipy import stats
from scipy.ndimage import uniform_filter1d
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from scipy.signal import medfilt, savgol_filter
import re
import itertools
import seaborn as sns

In [116]:
adata = sc.read_h5ad("sim_mixed_cna_goldstandard(1).h5ad")
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 500 × 2000
    obs: 'cell_type', 'simulated_cnvs', 'n_genes', 'n_counts'
    var: 'chromosome', 'start', 'end'

In [117]:
if 'X_pca' not in adata.obsm:
    print("Running PCA...")
    if adata.X.max() > 100:
        print("Log-normalizing...")
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
    sc.pp.pca(adata)
if 'neighbors' not in adata.uns:
    print("Running Neighbors...")
    sc.pp.neighbors(adata, n_pcs=min(30, adata.obsm['X_pca'].shape[1]))
if 'X_umap' not in adata.obsm:
    print("Running UMAP...")
    sc.tl.umap(adata)
if 'leiden_clusters' not in adata.obs:
    print("Running Leiden clustering...")
    sc.tl.leiden(adata, key_added='leiden_clusters')

Running PCA...
Running Neighbors...
Running UMAP...
Running Leiden clustering...


# Function definition
Here we have four important functions defined:
1. find_cluster_references: For each cluster (from adata.obs[cluster_key]) fits a 2-component GMM on per cell variance to pick out “diploid” reference cells.
2. detect_cnas_per_cluster: Tiles the genome into overlapping windows, computes per cluster log2 fold change vs. its reference cells, optionally smooths those LFCs, and calls gains/losses when a given fraction of windows crosses your thresholds.
3. extract_cna_segments: A helper function that takes the raw calls_chr plus the exact windows and sorted var used to build them and stitches consecutive windows of the same call into contiguous genomic segments, returning a per-cell list of (chromosome, start, end, genotype) dicts.
4. compute_mean_f1: Parses all CNV events from your
adata.obs['simulated_cnvs'], then for each (chromosome, sign) pair stacks true vs. predicted binary vectors and computes overall precision, recall, and F1 via precision_recall_fscore_support. Returns (precision, recall, f1_score)

In [74]:
def find_cluster_references(
    adata: sc.AnnData,
    cluster_key: str = 'leiden_clusters',
    layer: str | None = None,
    n_genes_subset: int = 3000,
    gmm_components: int = 2,
    random_state: int = 42,
    verbose: bool = False
) -> dict[str, list[str]]:
    """
    For each cluster in adata.obs[cluster_key], select a diploid reference
    via low‐variance cells. Returns { cluster: [cell, ...], ... }.
    """
    if cluster_key not in adata.obs:
        raise KeyError(f"{cluster_key!r} not in adata.obs")

    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.tocsr()

    nonmt = ~adata.var_names.str.upper().str.startswith('MT-')
    avail = np.where(nonmt)[0]
    if len(avail) >= n_genes_subset:
        rng = np.random.RandomState(random_state)
        subset_idx = rng.choice(avail, n_genes_subset, replace=False)
    else:
        subset_idx = avail if len(avail)>0 else np.arange(adata.n_vars)

    refs: dict[str, list[str]] = {}

    clusters = [str(x) for x in adata.obs[cluster_key].unique()]

    for cl in clusters:
        mask_cl = (adata.obs[cluster_key].astype(str).values == cl)
        idx_cl  = np.where(mask_cl)[0]
        if len(idx_cl) < 5:
            if verbose:
                print(f"Cluster {cl!r}: only {len(idx_cl)} cells → skipping")
            refs[cl] = []
            continue

        block = X[idx_cl[:, None], subset_idx]

        if scipy.sparse.issparse(block):
            block = block.toarray()

        vars_ = np.nanvar(block, axis=1, ddof=1).reshape(-1,1)

        gmm = GaussianMixture(n_components=gmm_components,
                              random_state=random_state)
        gmm.fit(vars_)
        lbl = gmm.predict(vars_)
        means = gmm.means_.flatten()
        dip = int(np.argmin(means))

        dip_idx   = idx_cl[lbl == dip]
        dip_cells = adata.obs_names[dip_idx].tolist()
        refs[cl]   = dip_cells

        if verbose:
            print(f"Cluster {cl!r}: {len(dip_cells)}/{len(idx_cl)} refs "
                  f"  (GMM means var={means.round(4)})")

    return refs

In [75]:
def detect_cnas_per_cluster(
    adata,
    cluster_key: str,
    cluster_refs: dict[str, list[str]],
    window_size: int = 100,
    min_genes_per_window: int = 10,
    smoothing_sigma: float = 1.0,
    lfc_threshold: float = 0.1,
    fraction_threshold: float = 0.2,
    layer: str | None = None
) -> tuple[pd.DataFrame, list[dict], pd.DataFrame]:
    """
    For each cluster in adata.obs[cluster_key], uses cluster_refs to compute
    smoothed per-window log2 fold-change and calls gains/losses by requiring
    that at least `fraction_threshold` of windows on a chromosome exceed
    +lfc_threshold (gain) or -lfc_threshold (loss).

    Returns
    -------
    calls_chr : pd.DataFrame
        DataFrame of shape (n_cells, n_chromosomes) with values in {-1, 0, +1}.
    """
    X = adata.layers[layer] if layer else adata.X
    if scipy.sparse.issparse(X):
        X = X.toarray()

    var = adata.var.copy()
    var['chrom'] = var['chromosome'].astype(str)
    var['start'] = pd.to_numeric(var['start'], errors='coerce')
    var = var.dropna(subset=['chrom','start'])
    def chr_key(c):
        c = str(c)
        return (int(c), '') if c.isdigit() else (np.inf, c)
    var['_idx'] = np.arange(var.shape[0])
    var = var.sort_values(['chrom','start'], key=lambda col: col.map(chr_key))
    sorted_idx = var['_idx'].values.astype(int)
    gene_chroms = var['chrom'].values

    Xs = X[:, sorted_idx]
    n_cells, n_win = Xs.shape

    windows = []
    window_expr = []
    for chrom in pd.unique(gene_chroms):
        idx_chr = np.where(gene_chroms == chrom)[0]
        step = max(1, window_size // 2)
        for i in range(0, len(idx_chr), step):
            block = idx_chr[i:i+window_size]
            if len(block) < min_genes_per_window:
                continue
            windows.append({'chrom': chrom, 'genes': block})
            window_expr.append(Xs[:, block].mean(axis=1))
    W = np.stack(window_expr, axis=1)

    chroms = sorted({w['chrom'] for w in windows}, key=chr_key)
    calls_chr = pd.DataFrame(0, index=adata.obs_names, columns=chroms, dtype=int)

    for cl in adata.obs[cluster_key].astype(str).unique():
        refs = cluster_refs.get(cl, [])
        if len(refs) < min_genes_per_window:
            continue
        mask_cl = (adata.obs[cluster_key].astype(str) == cl).values
        idx_cl = np.where(mask_cl)[0]
        mask_ref = adata.obs_names.isin(refs)
        idx_ref = np.where(mask_ref)[0]
        if len(idx_ref) == 0:
            continue
        ref_med = np.median(W[idx_ref, :], axis=0)
        ref_med = np.clip(ref_med, 1e-6, None)
        L = np.log2((W[idx_cl, :] + 1e-6) / ref_med[np.newaxis, :])
        if smoothing_sigma > 0:
            for chrom in chroms:
                win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
                if len(win_idxs) > 1:
                    L[:, win_idxs] = gaussian_filter1d(
                        L[:, win_idxs],
                        sigma=smoothing_sigma,
                        axis=1,
                        mode='reflect'
                    )
        for chrom in chroms:
            win_idxs = [i for i,w in enumerate(windows) if w['chrom']==chrom]
            sub = L[:, win_idxs]
            frac_gain = (sub > lfc_threshold).sum(axis=1) / len(win_idxs)
            frac_loss = (sub < -lfc_threshold).sum(axis=1) / len(win_idxs)
            cells = adata.obs_names[idx_cl]
            calls_chr.loc[cells[frac_gain >= fraction_threshold], chrom] = +1
            calls_chr.loc[cells[frac_loss >= fraction_threshold], chrom] = -1

    return calls_chr, windows, var

In [76]:
def extract_cna_segments(
    calls_chr: pd.DataFrame,
    windows: list[dict],
    var: pd.DataFrame
) -> pd.DataFrame:
    """
    Flatten per‐cell per‐window calls into a list of segments.
    Parameters
    ----------
    calls_chr : DataFrame
        index = cell barcodes, columns = chromosomes, values in {-1,0,1}
    windows : list of dict
        each dict has keys 'chrom' and 'genes' = array of sorted var indices
    var : DataFrame
        the same .var you used when building windows, with .loc[first,'start'] and .loc[last,'end']
    Returns
    -------
    segs : DataFrame with columns
        ['cell', 'chromosome', 'start', 'end', 'genotype']
    """
    records = []

    win_meta = []
    for w in windows:
        chrom = w['chrom']
        gene_inds = w['genes']
        gene_names = var.index[gene_inds]
        st = var.loc[gene_names, 'start'].min()
        en = var.loc[gene_names, 'end'].max()
        win_meta.append((chrom, st, en))

    for cell in calls_chr.index:

        for win_i, (chrom, st, en) in enumerate(win_meta):
            call = calls_chr.at[cell, chrom]
            if call == 0:
                continue
            records.append({
                'cell':       cell,
                'chromosome': chrom,
                'start':      st,
                'end':        en,
                'genotype':   'gain' if call==1 else 'loss'
            })

    return pd.DataFrame(records)

# Example Usage
This pipeline first identifies “diploid” reference cells in each Leiden cluster via a low variance GMM (find_cluster_references), then tiles the genome into overlapping windows and computes per-cluster log fold changes versus those references to call gains and losses (detect_cnas_per_cluster). The raw binary calls are stitched into segments with (extract_cna_segments), producing a table of (cell, chromosome, start, end, genotype). A final merge with the Leiden cluster labels produces a per cell segment table that can be compared against the adata.obs['simulated_cnvs'] ground truth via precision/recall/F1 metrics, enabling systematic tuning of window size, smoothing parameters, and threshold values.

In [118]:
cluster_refs = find_cluster_references(
    adata,
    cluster_key='leiden_clusters',
    layer=None,
    n_genes_subset=2000,
    gmm_components=2,
    random_state=42,
    verbose=True
)

Cluster '2': 51/100 refs   (GMM means var=[6.0441 5.6932])
Cluster '1': 86/150 refs   (GMM means var=[4.896  5.1415])
Cluster '0': 143/250 refs   (GMM means var=[8.0814 7.697 ])


In [119]:
calls_chr, windows, var_sorted = detect_cnas_per_cluster(
        adata,
        cluster_key='leiden_clusters',
        cluster_refs=cluster_refs,
        window_size=100,
        min_genes_per_window=10,
        smoothing_sigma=1.0,
        lfc_threshold=0.01,
        fraction_threshold=0.5,
        layer=None
    )

In [120]:
cna_segments = extract_cna_segments(calls_chr, windows, var_sorted)

In [121]:
cna_segments = (
    cna_segments
    .merge(
        adata.obs[['leiden_clusters']],
        left_on='cell',
        right_index=True
    )
)
cna_segments.head()

,cell,chromosome,start,end,genotype,leiden_clusters
0,Cell1,chr1,1,100000.0,loss,2
1,Cell1,chr1,50001,150000.0,loss,2
2,Cell1,chr1,100001,200000.0,loss,2
3,Cell1,chr1,150001,250000.0,loss,2
4,Cell1,chr1,200001,300000.0,loss,2


In [122]:
def compute_mean_f1(calls_chr, adata):
    """
    calls_chr : pd.DataFrame, index=cells, cols=chromosomes, values in {-1,0,+1}
    adata.obs['simulated_cnvs']: either simple 'gain'/'none' or full "chr:start-end (CN #)" strings.
    """
    sim = adata.obs['simulated_cnvs'].astype(str).fillna('')
    uniques = set(sim.unique())

    if uniques <= {'gain','none'}:
        y_true = (sim == 'gain').astype(int).values

        y_pred = (calls_chr.values == 1).any(axis=1).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_true, y_pred,
            average='binary', zero_division=0
        )
        return float(f1)

    ev = []
    for s in sim.values:
        for part in s.split(','):
            m = re.match(r'(\w+):\d+-\d+\s+\(CN\s+(\d+)\)', part.strip())
            if not m:
                continue
            chrom, cn = m.group(1), int(m.group(2))
            sign = +1 if cn > 2 else (-1 if cn < 2 else 0)
            if sign != 0:
                ev.append((chrom, sign))
    ev = list(set(ev))
    if not ev:
        raise ValueError("No CNV events parsed from 'simulated_cnvs'")

    ys, ps = [], []
    for chrom, sign in ev:
        y = sim.str.contains(rf'\b{chrom}:').astype(int).values
        p = (calls_chr[chrom].values == sign).astype(int)
        ys.append(y)
        ps.append(p)

    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    _, _, f1, _ = precision_recall_fscore_support(
        y_true, y_pred,
        average='binary', zero_division=0
    )
    return float(f1)

In [123]:
records = []
for w, s, lfc, frac in itertools.product(
        [50,100,200],        # window_size
        [1.0,2.0,4.0],       # smoothing_sigma
        [0.05,0.1,0.2],      # lfc_threshold
        [0.1,0.2,0.3]        # fraction_threshold
    ):

    result = detect_cnas_per_cluster(
        adata,
        cluster_key='leiden_clusters',
        cluster_refs=cluster_refs,
        window_size=w,
        smoothing_sigma=s,
        lfc_threshold=lfc,
        fraction_threshold=frac,
        min_genes_per_window=10,
        layer=None
    )

    if isinstance(result, tuple):
        calls_chr = result[0]
    else:
        calls_chr = result

    mean_f1 = compute_mean_f1(calls_chr, adata)

    records.append({
        'window_size':     w,
        'smoothing_sigma': s,
        'lfc_threshold':   lfc,
        'fraction_threshold': frac,
        'mean_f1':         mean_f1
    })

df = pd.DataFrame(records)
df_best = df.sort_values('mean_f1', ascending=False).reset_index(drop=True)
print(df_best.head(5))

   window_size  smoothing_sigma  lfc_threshold  fraction_threshold   mean_f1
0           50              1.0           0.05                 0.2  0.743624
1           50              2.0           0.05                 0.1  0.717514
2          100              1.0           0.05                 0.2  0.714286
3          100              1.0           0.05                 0.1  0.708039
4           50              2.0           0.05                 0.2  0.702398
